In [6]:
import dgl_ptm
import os
os.environ["DGLBACKEND"] = "pytorch"

In [2]:
# Create an instance of the model class
model = dgl_ptm.PovertyTrapModel(model_identifier='test')

# Set the model and simulation parameters
# Using DEFAULT values
# model.set_model_parameters(default=True)

# OR
# using a CONFIG file
model.set_model_parameters(default=True, parameterFilePath='../dgl_ptm/dgl_ptm/config.yaml')

# Initialize the model
model.initialize_model()

In [4]:
model.steering_parameters['npath']

'./test/agent_data.zarr'

In [5]:
model.run()

performing step 1 of 20
creating bidirectional link between nodes 1 (src) and tensor([90]) (dst)
created 1 of 1 links requested
performing step 2 of 20
creating bidirectional link between nodes 45 (src) and tensor([46]) (dst)
created 1 of 1 links requested
performing step 3 of 20
creating bidirectional link between nodes 27 (src) and tensor([2]) (dst)
created 1 of 1 links requested
performing step 4 of 20
creating bidirectional link between nodes 42 (src) and tensor([26]) (dst)
created 1 of 1 links requested
performing step 5 of 20
creating bidirectional link between nodes 23 (src) and tensor([79]) (dst)
created 1 of 1 links requested
performing step 6 of 20
dst node 91 is an end node
no FoF link possible for src dst nodes 78 and 91.
created 0 of 1 links requested
performing step 7 of 20
creating bidirectional link between nodes 33 (src) and tensor([7]) (dst)
created 1 of 1 links requested
performing step 8 of 20
creating bidirectional link between nodes 6 (src) and tensor([78]) (dst)


RuntimeError: execution of step failed for step 10